In [1]:
import numpy as np
import math

In [2]:
np.set_printoptions(formatter={'float_kind':"{:.2f}".format})

# Eliminare gaussiana

In [3]:
n = 7

A = np.random.uniform(-1, 1, size=(n, n))
b = np.random.uniform(-1, 1, size=(n, 1))
# print(A)
# print(b)

Ae = np.concatenate([A, b], axis=1)
print(Ae)
print(Ae[0:, 0])

for i in range(n-1):
    p = i + np.argmax(np.abs(Ae[i:, i]))
    if math.isclose(float(Ae[p, i]), 0):
        assert False
    
    if p != i:
        Ae[[p, i]] = Ae[[i, p]]

    for j in range(i+1, n):
        m = Ae[j, i] / Ae[i, i]
        Ae[j] -= m * Ae[i]

print(Ae)
x = np.zeros(n)
for i in range(n-1, -1, -1):
    x[i] = (Ae[i, n] - np.sum(Ae[i, i+1:n] * x[i+1:n])) / Ae[i, i]
x = x.reshape(-1, 1)

print(x)
print(np.linalg.solve(A, b))

[[0.27 0.87 -0.98 -0.76 -0.06 0.27 0.41 -0.42]
 [0.38 -0.05 -0.97 -0.84 0.52 -0.37 0.61 0.63]
 [-0.34 -0.80 0.40 0.94 0.38 0.98 -0.82 0.99]
 [0.85 0.86 -0.80 -0.95 -0.93 0.87 0.21 0.85]
 [0.53 -0.81 -0.71 -0.60 0.23 -0.55 0.50 -0.27]
 [0.49 0.60 -0.42 0.27 -0.80 -0.39 0.30 0.12]
 [0.38 0.27 0.52 0.82 -0.07 -0.07 0.70 -0.04]]
[0.27 0.38 -0.34 0.85 0.53 0.49 0.38]
[[0.85 0.86 -0.80 -0.95 -0.93 0.87 0.21 0.85]
 [0.00 -1.35 -0.21 0.00 0.82 -1.10 0.36 -0.80]
 [0.00 0.00 0.90 1.25 0.28 -0.36 0.58 -0.35]
 [0.00 0.00 0.00 0.80 -0.21 -0.97 0.19 -0.42]
 [0.00 0.00 0.00 0.00 1.04 0.01 0.88 -1.01]
 [0.00 0.00 0.00 0.00 0.00 2.18 -0.85 1.61]
 [0.00 0.00 0.00 0.00 0.00 0.00 -0.21 1.55]]
[[9.35]
 [3.27]
 [1.85]
 [0.03]
 [5.31]
 [-2.13]
 [-7.40]]
[[9.35]
 [3.27]
 [1.85]
 [0.03]
 [5.31]
 [-2.13]
 [-7.40]]


# Descompunere LUP

In [9]:
def lup_decomp(A):
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    A = A.copy()
    
    P = np.eye(n)
    
    for k in range(n-1):
        i = k + np.argmax(np.abs(A[k:, k]))
        if i != k:
            A[[i, k]] = A[[k, i]]
            P[[i, k]] = P[[k, i]]
        
        lin = np.s_[k+1:n]
        
        A[lin, k] /= A[k, k]
        A[lin, lin] -= np.matmul(A[lin, [k]], A[[k], lin])
    
    L = np.tril(A)
    np.fill_diagonal(L, 1)
    
    U = np.triu(A)

    return L, U, P


def lup_solve(A, b):
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    
    L, U, P = lup_decomp(A)
    
    Pb = np.matmul(P, b)
    
    y = np.zeros(n)
    for i in range(n):
        y[i] = (Pb[i, 0] - np.sum(L[i, :i] * y[:i])) / L[i, i]
    y = y.reshape(-1, 1)
    
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i, 0] - np.sum(U[i, i:] * x[i:])) / U[i, i]
    x = x.reshape(-1, 1)

    return x

In [8]:
n = 7

A = np.random.uniform(-1, 1, size=(n, n))
b = np.random.uniform(-1, 1, size=(n, 1))

x = lup_solve(A, b)

print(x)
print(np.linalg.solve(A, b))

[[2.23]
 [-3.87]
 [-2.75]
 [-1.23]
 [-1.55]
 [0.28]
 [-3.23]]
[[2.23]
 [-3.87]
 [-2.75]
 [-1.23]
 [-1.55]
 [0.28]
 [-3.23]]


# Descompunere Cholesky

$$
\begin{aligned}
Ax&=b \\
A&=LL^T \\
Ly&=b \\
L^Tx&=y \\
\end{aligned}
$$

In [10]:
def cholesky_decomp(A):
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    
    # check A is positively defined just in case
    for i in range(n):
        a = A[:i+1, :i+1]
        assert np.linalg.det(a) > 0

    L = np.zeros_like(A)
    for k in range(n):
        L[k, k] = np.sqrt(A[k, k] - np.sum(L[k, :k]**2))
        for i in range(k+1, n):
            L[i, k] = (A[i, k] - np.sum(L[i, :k]*L[k, :k])) / L[k, k]
    return L


def cholesky_solve(A, b):
    assert len(A.shape) == 2
    assert A.shape[0] == A.shape[1]
    n = A.shape[0]
    
    L = cholesky_decomp(A)

    y = np.zeros(n)
    for i in range(n):
        y[i] = (b[i, 0] - np.sum(L[i, :i] * y[:i])) / L[i, i]
    y = y.reshape(-1, 1)
    
    Lt = np.transpose(L)
    
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (y[i, 0] - np.sum(Lt[i, i:] * x[i:])) / Lt[i, i]
    x = x.reshape(-1, 1)

    return x

In [12]:
n = 7

A = np.random.uniform(0, 1, size=(n, n))
A = (A + np.transpose(A)) / 2  # ensure A is hermitian
A += n*np.eye(n)  # ensure A is positively defined

b = np.random.uniform(-1, 1, size=(n, 1))

x = cholesky_solve(A, b)

print(x)
print(np.linalg.solve(A, b))

[[-0.01]
 [0.00]
 [-0.02]
 [0.11]
 [0.12]
 [0.05]
 [-0.01]]
[[-0.01]
 [0.00]
 [-0.02]
 [0.11]
 [0.12]
 [0.05]
 [-0.01]]
